In [105]:
import numpy as np
import random
import math

# Funciones

## Test en binarios

In [136]:
class MaxOnes():
    def f(self, x):
        suma = 0
        for i in x:
            suma += i
        return suma

## Reales

Se multiplican las funciones por -1 para convertirlas a problemas de maximización
    
Cada dimensión se codifica en vectores de 16 bits que se concatenan uno tras otro
para formar el código genético.

La función de crecimiento partirá el código genético en vectores de 16 bits, que
serán convertidos a enteros de 16 bits y luego normalizados dentro de los límites

Ejemplo para Rastrigin (-10.5,10.5):

    xi = (enterode16bits)*21/(2**16) - 10.5


In [545]:
class Rastrigin():
    '''[-10.5,10.5]'''
    bits = 16
    def crecimiento(self, codGenetico):
        dimensiones = int(len(codGenetico)/self.bits)
        x=[]
        for d in range(0,dimensiones):
            xi = 0
            for i in range(0,self.bits):
                xi += (codGenetico[i+self.bits*d]*2**i)
            x.append(xi*21/(2**16) - 10.5)
        return x
    ## Sea x un vector de reales
    def fitness(self, x):
        suma = 10*len(x)
        for xi in x:
            suma += (xi**2 - 10*math.cos(2*math.pi*xi))
        return -suma
    def f(self, codGenetico):
        x = self.crecimiento(codGenetico)
        return self.fitness(x)

In [554]:
class Schwefel():
    '''[-500,500]'''
    bits = 16
    def crecimiento(self, codGenetico):
        dimensiones = int(len(codGenetico)/self.bits)
        x=[]
        for d in range(0,dimensiones):
            xi = 0
            for i in range(0,self.bits):
                xi += (codGenetico[i+self.bits*d]*2**i)
            x.append(xi*1000/(2**16) - 500)
        return x
    ## Sea x un vector de reales
    def fitness(self, x):
        suma = 418.9829*len(x)
        for xi in x:
            suma -= (xi*math.sin(math.sqrt(abs(xi))))
        return -suma
    def f(self, codGenetico):
        x = self.crecimiento(codGenetico)
        return self.fitness(x)

In [560]:
class Griewank():
    '''[-600,600]'''
    bits = 16
    def crecimiento(self, codGenetico):
        dimensiones = int(len(codGenetico)/self.bits)
        x=[]
        for d in range(0,dimensiones):
            xi = 0
            for i in range(0,self.bits):
                xi += (codGenetico[i+self.bits*d]*2**i)
            x.append(xi*1200/(2**16) - 600)
        return x
    ## Sea x un vector de reales
    def fitness(self, x):
        suma = 0
        prod = 1
        d = 0
        for xi in x:
            d += 1
            suma = suma + xi**2/4000;
            prod = prod * math.cos(xi/math.sqrt(d));
        return -(suma - prod + 1)
    def f(self, codGenetico):
        x = self.crecimiento(codGenetico)
        return self.fitness(x)

In [567]:
class Rosenbrock():
    '''[-5,10]'''
    bits = 16
    def crecimiento(self, codGenetico):
        dimensiones = int(len(codGenetico)/self.bits)
        x=[]
        for d in range(0,dimensiones):
            xi = 0
            for i in range(0,self.bits):
                xi += (codGenetico[i+self.bits*d]*2**i)
            x.append(xi*15/(2**16) - 5)
        return x
    ## Sea x un vector de reales
    def fitness(self, x):
        suma = 0
        for i in range(0,len(x)-1):
            suma += 100*(x[i+1]-x[i]**2)**2 + (x[i]-1)**2;
        return -suma
    def f(self, codGenetico):
        x = self.crecimiento(codGenetico)
        return self.fitness(x)

# Algoritmos

In [358]:
# f es el fitness, x el código genético
class Individuo():
    x = []
    f = 0
class IndividuoHAEA():
    pcruce=0.5
    pmutacion=0.5
    x = []
    f = 0
    def iniciarOperadores(self):
        pcrucetemp = random.random()
        pmutaciontemp = random.random()
        suma = pcrucetemp + pmutaciontemp
        self.pcruce = pcrucetemp/suma
        self.pmutacion = pmutaciontemp/suma
    def normalizarOperadores(self):
        suma = self.pcruce + self.pmutacion
        self.pcruce = self.pcruce/suma
        self.pmutacion = self.pmutacion/suma
    def premiarCruce(self):
        self.pcruce = self.pcruce + random.random()
        self.normalizarOperadores()
    def premiarMutacion(self):
        self.pmutacion = self.pmutacion + random.random()
        self.normalizarOperadores()

In [256]:
class AlgoritmoGenetico():
    def iniciarPoblacion(self, n, bits, funcion):
        poblacion = []
        for i in range(0,n):
            individuo = Individuo()
            individuo.x = np.random.choice([0, 1], size=(bits)).tolist()
            individuo.f = funcion.f(individuo.x)
            poblacion.append(individuo)
        return poblacion
    def selectorRuleta(self, individuos):
        boletas = []
        conjunto = []
        padres = []
        sumaFitness = 0
        for individuo in individuos:
            sumaFitness += individuo.f
        for individuo in individuos:
            boletas.append(int(len(individuos)*10*individuo.f/sumaFitness)+1)
        for i in range(0,len(individuos)):
            for j in range(0,boletas[i]):
                conjunto.append(i)
        np.random.shuffle(conjunto)
        for i in range(0,len(individuos)):
            padres.append(individuos[conjunto[int(np.random.uniform(0,len(conjunto)))]])
        return padres
    def selectorTorneo(self, individuos):
        padres = []
        for i in range(0,len(individuos)):
            a = np.arange(0,len(individuos)).tolist()
            posCompetidores = np.random.choice(a, size=(4)).tolist()
            posMejor = posCompetidores[0]
            for i in posCompetidores:
                if individuos[i].f > individuos[posMejor].f:
                    posMejor = i
            padres.append(individuos[posMejor])
        return padres
    def selectorUniversalEstocastico(self, individuos):
        individuos = sorted(individuos, key=lambda individuo: individuo.f)
        k = len(individuos)
        sumaF = 0
        for i in individuos:
            sumaF += i.f
        d = sumaF/k
        pi = np.random.uniform(0,d)
        padres = []
        i = 0
        for p in range(0,k):
            while (i < k-1) and (individuos[i].f < pi + p*d):
                i += 1
            padres.append(individuos[i])
        return padres
    def selectorElitista(self, individuos):
        individuos = sorted(individuos, key=lambda individuo: individuo.f, reverse = True)
        elite = int(len(individuos)*0.1)
        claseMedia = int(len(individuos)*0.8)
        padresElite = []
        padresClaseMedia = []
        for i in range(0,int(len(individuos)/2)):
            padresElite.append(individuos[int(np.random.uniform(0,elite))])
            padresClaseMedia.append(individuos[int(np.random.uniform(elite,claseMedia))])
        diferencia = len(individuos) - (len(padresElite) + len(padresClaseMedia))
        while diferencia > 0:
            padresElite.append(individuos[int(np.random.uniform(0,elite))])
            diferencia -= 1
        return padresElite + padresClaseMedia
    def cruceYMutacionGaussianaPoblacion(self, padres, probCruce, mediaMutacion, sigmaMutacion, funcion):
        a = np.arange(0,len(padres))
        np.random.shuffle(a)
        a = a.reshape(int(len(padres)/2),2).tolist()
        hijos = []
        for pospareja in a:
            if random.random() < probCruce:
                codGenPareja = [padres[pospareja[0]].x, padres[pospareja[1]].x]
                temp = self.cruce(codGenPareja)
                for i in range(0,len(temp)):
                    hijo = Individuo()
                    hijo.x = self.mutacionGaussiana(temp[i], mediaMutacion, sigmaMutacion)
                    hijo.f = funcion.f(hijo.x)
                    hijos.append(hijo)
            else:
                hijos += [padres[pospareja[0]], padres[pospareja[1]]]
        return hijos
    def cruce(self, codGenPareja):
        punto = int(np.random.uniform(1,len(codGenPareja[0])))
        hijosPareja = []
        hijosPareja.append(codGenPareja[0][:punto]+codGenPareja[1][punto:])
        hijosPareja.append(codGenPareja[1][:punto]+codGenPareja[0][punto:])
        return hijosPareja
    def mutacionGaussiana(self, codGen, media, sigma):
        cantidad = int(random.gauss(media,sigma))
        cantidad = cantidad if (cantidad > 0) else 0
        cantidad = cantidad if (cantidad < len(codGen)) else len(codGen)
        a = np.arange(0,len(codGen)).tolist()
        posMutaciones = np.random.choice(a, size=(cantidad), replace= False).tolist()
        for i in posMutaciones:
            codGen[i] = 0 if (codGen[i] == 1) else 1
        return codGen
    def reemplazoGeneracional(self, padres, hijos):
        return hijos
        
    

In [355]:
def resultadosAGC(poblacion):
    res = sorted(poblacion, key=lambda individuo: individuo.f, reverse = True)
    sumaF = 0
    for i in poblacion:
        sumaF += i.f
    media = sumaF/len(poblacion)
    var = 0
    for i in poblacion:
        var += (i.f - media)**2
    var = var/len(poblacion)
    sd = math.sqrt(var)
    print([res[0].f,res[99].f, media])
def resultadosHAEA(poblacion):
    res = sorted(poblacion, key=lambda individuo: individuo.f, reverse = True)
    sumaF = 0
    sumaPCruce = 0
    sumaPMutacion = 0
    for i in poblacion:
        sumaF += i.f
        sumaPCruce += i.pcruce
        sumaPMutacion += i.pmutacion
    mediaPCruce = sumaPCruce/len(poblacion)
    mediaPMutacion = sumaPMutacion/len(poblacion)
    media = sumaF/len(poblacion)
    var = 0
    for i in poblacion:
        var += (i.f - media)**2
    var = var/len(poblacion)
    sd = math.sqrt(var)
    print([res[0].f,res[99].f, media,"--",mediaPCruce,mediaPMutacion])

In [520]:
class HAEA():
    def iniciarPoblacion(self, n, bits, funcion):
        poblacion = []
        for i in range(0,n):
            individuo = IndividuoHAEA()
            individuo.x = np.random.choice([0, 1], size=(bits)).tolist()
            individuo.f = funcion.f(individuo.x)
            individuo.iniciarOperadores()
            poblacion.append(individuo)
        return poblacion
    def operar(self, individuo, poblacion, mediaMutacion, sigmaMutacion, funcion):
        selector = random.random()
        individuoQuePasa = IndividuoHAEA()
        if selector <= individuo.pcruce:
            a = np.arange(0,len(poblacion)).tolist()
            posCompetidores = np.random.choice(a, size=(4)).tolist()
            posMejor = posCompetidores[0]
            for i in posCompetidores:
                if poblacion[i].f > poblacion[posMejor].f:
                    posMejor = i
            codGenHijos = self.cruce([individuo.x, poblacion[posMejor].x])
            f0 = funcion.f(codGenHijos[0])
            f1 = funcion.f(codGenHijos[1])
            mejorHijo = IndividuoHAEA()
            if f0 > f1:
                mejorHijo.x = codGenHijos[0]
                mejorHijo.f = f0
            else:
                mejorHijo.x = codGenHijos[1]
                mejorHijo.f = f1
            
            if mejorHijo.f >= individuo.f:
                individuoQuePasa = mejorHijo
                individuoQuePasa.premiarCruce()
            else:
                individuoQuePasa = individuo
                individuoQuePasa.premiarMutacion()
        else:
            hijo = IndividuoHAEA()
            hijo.x = self.mutacionGaussiana(individuo.x, mediaMutacion, sigmaMutacion)
            hijo.f = funcion.f(hijo.x)
            if hijo.f >= individuo.f:
                individuoQuePasa = hijo
                individuoQuePasa.premiarMutacion()
            else:
                individuoQuePasa = individuo
                individuoQuePasa.premiarCruce()
        return individuoQuePasa
    def cruce(self, codGenPareja):
        punto = int(np.random.uniform(1,len(codGenPareja[0])))
        hijosPareja = []
        hijosPareja.append(codGenPareja[0][:punto]+codGenPareja[1][punto:])
        hijosPareja.append(codGenPareja[1][:punto]+codGenPareja[0][punto:])
        return hijosPareja
    def mutacionGaussiana(self, codGen, media, sigma):
        cantidad = int(random.gauss(media,sigma))
        cantidad = cantidad if (cantidad > 0) else 0
        cantidad = cantidad if (cantidad < len(codGen)) else len(codGen)
        a = np.arange(0,len(codGen)).tolist()
        posMutaciones = np.random.choice(a, size=(cantidad), replace= False).tolist()
        for i in posMutaciones:
            codGen[i] = 0 if (codGen[i] == 1) else 1
        return codGen

# Experimentos

In [572]:
####################### AG Canónico
n = 100
bits = 160
iteraciones = 100
ag = AlgoritmoGenetico()
funcion = Rastrigin()
poblacion = ag.iniciarPoblacion(n, bits, funcion)
for i in range(0, iteraciones):
    padres = ag.selectorElitista(poblacion)
    hijos = ag.cruceYMutacionGaussianaPoblacion(padres, 0.8, 1,1, funcion)
    poblacion = ag.reemplazoGeneracional(padres, hijos)

    resultadosAGC(poblacion)

[-206.98581337856987, -687.6106420903092, -390.27366287925327]
[-109.35619079745601, -485.7476265235068, -314.50138474108644]
[-103.35603344293634, -398.4860690494658, -242.95931870800467]
[-87.36655119944314, -337.2727628558734, -178.57600634798217]
[-87.36655119944314, -307.0840231255734, -139.98278062966526]
[-68.70447259918387, -222.6739839744299, -120.29783332287526]
[-68.70447259918387, -234.84757371707897, -104.82977986344608]
[-67.98114418345438, -212.57107484188668, -92.60336908288775]
[-57.889670500353674, -189.48305237277904, -86.71572102019826]
[-47.749378955283575, -146.66143574161384, -74.36798429455403]
[-47.749378955283575, -174.82211190603851, -68.11832817628483]
[-39.86951534298795, -221.67535856192546, -67.7090520503894]
[-39.86951534298795, -171.56983827816578, -60.947265708666194]
[-38.83839236435567, -167.9128096778487, -54.45180071305068]
[-28.888455852038557, -198.63278747219263, -51.46274958128259]
[-28.888455852038557, -163.85351363605935, -47.81258599384722]


In [573]:
####################### HAEA
n = 100
bits = 160
iteraciones = 100
haea = HAEA()
funcion = Rastrigin()
poblacion = haea.iniciarPoblacion(n, bits, funcion)
for i in range(0, iteraciones):
    nuevaGeneracion = []
    j=0
    for individuo in poblacion:
        nuevaGeneracion.append(haea.operar(individuo,poblacion,1,3,funcion))
        j+=1
    poblacion = nuevaGeneracion.copy()
    resultadosHAEA(poblacion)

[-175.47802562372948, -668.9743691352212, -406.87676622960333, '--', 0.5403989806794602, 0.45960101932053976]
[-149.41722966216057, -653.5561400343797, -359.89307951648254, '--', 0.5419261914860384, 0.45807380851396146]
[-149.41722966216057, -653.5561400343797, -310.85624406177556, '--', 0.5176519198295003, 0.4823480801704995]
[-135.5267660925928, -653.5561400343797, -267.18987618926707, '--', 0.5361680527026295, 0.4638319472973702]
[-100.93140202376775, -613.785101469425, -220.3753756584509, '--', 0.5531660299418534, 0.4468339700581467]
[-100.93140202376775, -613.785101469425, -187.5340285875434, '--', 0.5298594978545723, 0.4701405021454277]
[-100.93140202376775, -596.6336885622128, -166.11172976505392, '--', 0.5446473761332973, 0.45535262386670267]
[-80.922888283391, -401.17397108605724, -141.86833822215428, '--', 0.5673494224320774, 0.43265057756792275]
[-77.07235116514278, -296.61084822996446, -128.1211676435522, '--', 0.5422005133716472, 0.4577994866283528]
[-62.16924721476582, -2

[-11.22296966154839, -13.03787895419038, -11.341633771148178, '--', 0.5584826419626041, 0.44151735803739567]
[-11.205800900047638, -13.03787895419038, -11.331875054600363, '--', 0.5447698594339246, 0.4552301405660755]
[-11.205800900047638, -13.03787895419038, -11.328520718414397, '--', 0.5614706664567963, 0.4385293335432037]
[-11.205800900047638, -13.03787895419038, -11.321928313812204, '--', 0.5425736384212109, 0.45742636157878896]
[-11.202248472612315, -13.03787895419038, -11.318630865167002, '--', 0.5524935391533127, 0.4475064608466874]
[-11.174391888621901, -13.03787895419038, -11.310764388356409, '--', 0.5336491311854542, 0.46635086881454585]
[-11.169294817107005, -13.03787895419038, -11.298882733406305, '--', 0.539840751169491, 0.46015924883050885]
[-11.169294817107005, -13.03787895419038, -11.29476696022911, '--', 0.5497509946939434, 0.45024900530605644]
[-11.163445660908959, -13.03787895419038, -11.289493730758984, '--', 0.5906515052575948, 0.4093484947424051]
[-11.161384081996

In [476]:
n = 100
bits = 160
iteraciones = 100
haea = HAEA()
funcion = Rastrigin()
poblacion = haea.iniciarPoblacion(n, bits, funcion)

In [553]:
2**16

65536